In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [25]:
data=pd.read_excel("train (1).xlsx")

In [26]:
data.head()

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1


In [27]:
data["rating"].unique()

array([-1,  1,  0], dtype=int64)

In [28]:
import torch
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"✅ الجهاز المستخدم هو: {device}")


model_id = 'Omartificial-Intelligence-Space/Arabic-MiniLM-L12-v2-all-nli-triplet'


try:
    model = SentenceTransformer(model_id).to(device)
    print("✅ تم تحميل النموذج بنجاح ونقله إلى CUDA/GPU.")
except Exception as e:
    print(f"❌ حدث خطأ أثناء التحميل: {e}")
    model = SentenceTransformer(model_id) # محاولة التحميل على CPU


✅ الجهاز المستخدم هو: cuda
✅ تم تحميل النموذج بنجاح ونقله إلى CUDA/GPU.


In [29]:
X=data["review_description"]

In [30]:
Y=np.array(data["rating"])+1


In [31]:
print(Y)

[0 2 0 ... 0 0 2]


In [32]:
with torch.no_grad(): # لتقليل استهلاك الذاكرة وعدم حساب التدرجات (Gradients)
    X = model.encode(X, convert_to_tensor=True, device=device)


In [39]:
X=X.cpu()

In [40]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

In [47]:
import xgboost as xgb
xx=xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, n_jobs=-1)
xx.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, ...)

In [48]:
from sklearn.metrics import classification_report
preds_xx=xx.predict(x_test)
print(classification_report(y_test,preds_xx))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77      2298
           1       0.29      0.01      0.01       288
           2       0.83      0.89      0.86      3822

    accuracy                           0.81      6408
   macro avg       0.63      0.56      0.55      6408
weighted avg       0.78      0.81      0.79      6408



In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
clss=LogisticRegression(max_iter=500)
clss.fit(x_train,y_train)

LogisticRegression(max_iter=500)

In [43]:
from sklearn.metrics import classification_report
preds=clss.predict(x_test)
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.77      0.76      0.77      2298
           1       0.25      0.02      0.03       288
           2       0.83      0.90      0.86      3822

    accuracy                           0.81      6408
   macro avg       0.62      0.56      0.55      6408
weighted avg       0.78      0.81      0.79      6408

